# 📚 StudyBuddy

**Creator:** Muhammad Bin Zohaib, Age 12, Batch 51, Lives in Karachi

## Overview

**StudyBuddy** is a student-focused AI assistant designed to help you with:
- 📖 Studying
- 📝 Exam preparation
- 🔍 Research on topics
- 💪 Staying motivated

It provides valuable academic assistance while keeping a friendly, encouraging tone.

## Features

- **Short-! Memory (🧠):** Remembers recent turns in the conversation to provide more relevant, context-aware answers.
- **Research on Demand (🔎):**  
  Just say "research on this" and StudyBuddy will use the **Tavily API** to fetch up-to-date information, along with a helpful link.
- **Jokes (🤣) and Quotes (💬):**  
  - Say "joke" for a quick laugh: StudyBuddy taps into the **icanhazdadjoke API** for a random, funny joke.
  - Say "quote" for inspiration: StudyBuddy offers uplifting quotes to keep you going.
- **General Study Help (💡):**  
  If it’s not a research request, joke, quote, or quit command, StudyBuddy calls on **Gemini (Google Generative AI)** to answer your academic questions, explain concepts, and guide your learning.
- **Quit Command (🚪):**  
  Type "quit" to receive a final motivational message before StudyBuddy ends the session.

## How It Works

1. **User Input (👩‍💻):**  
   You ask a question or give a command.
   
2. **Keyword Check (🔑):**
   - "research on this": Uses Tavily for real-time info.
   - "joke": Fetches a random joke.
   - "quote": Provides an encouraging quote.
   - "quit": Ends the conversation with motivation.
   - Otherwise, uses Gemini for a helpful, context-driven response.
   
3. **Short-Term Memory (🧠):**  
   StudyBuddy remembers your last 5 turns to keep the conversation contextually meaningful.

4. **Response (🤝):**  
   StudyBuddy returns a thoughtful, study-focused answer, tailored to help you learn and grow.

## Why It's Useful

- **For Students (🎓):** Perfect if you’re looking for quick academic explanations, study tips, exam prep strategies, or a little motivation.
- **Real-Time Research (🌐):** Need current info for an assignment or project? StudyBuddy has you covered.
- **Motivation & Fun (✨):** StudyBuddy’s jokes and quotes help break the monotony and keep your spirits high!

With StudyBuddy by your side, every study session can be more productive, informative, and enjoyable!


In [2]:
# Installing Required Libraries
!pip install langchain_google_genai tavily-python langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.2 MB/s eta 0:00:00


In [10]:
# Required libraries
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from google.colab import userdata
from tavily import TavilyClient
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
import json
import requests
import random
from langchain_google_genai import ChatGoogleGenerativeAI

def get_quote():
    return "“The beautiful thing about learning is that nobody can take it away from you.” – B.B. King"

def get_motivational_quote():
    return "Keep pushing forward in your studies, every bit of effort adds to your success!"

# Q&A style jokes list
jokes_qa = [
    {"question": "Why was the math book sad?", "answer": "Because it had too many problems."},
    {"question": "What do you call cheese that isn't yours?", "answer": "Nacho cheese!"},
    {"question": "Why don't scientists trust atoms?", "answer": "Because they make up everything!"},
    {"question": "Why did the student eat his homework?", "answer": "Because the teacher said it was a piece of cake!"},
    {"question": "What do you call a bear with no teeth?", "answer": "A gummy bear!"}
]

def get_joke_qa():
    joke = random.choice(jokes_qa)
    return f"Q: {joke['question']}\nA: {joke['answer']}"

study_tips = [
    "Try using the Pomodoro Technique: study for 25 minutes, then rest for 5.",
    "Teach the material to someone else—it helps you understand it better.",
    "Break big tasks into small, manageable steps to avoid feeling overwhelmed.",
    "Use flashcards for quick review sessions throughout the day.",
    "Set specific, achievable study goals before you start."
]

def get_study_tip():
    return random.choice(study_tips)

facts = [
    "Did you know? The Eiffel Tower can be 15 cm taller during hot days due to thermal expansion.",
    "Honey never spoils. Archaeologists have found edible honey in ancient Egyptian tombs.",
    "Your brain isn't fully developed until around age 25.",
    "Bananas are berries, but strawberries are not.",
    "Sharks have been around longer than trees."
]

def get_fact():
    return random.choice(facts)

subject_prompts = {
    "math": "For math: Try to understand the concept rather than just memorizing formulas!",
    "history": "For history: Creating a timeline of events can help you remember what happened when.",
    "science": "For science: Relate concepts to real-life examples to make them more memorable.",
    "english": "For English: Reading a variety of texts improves comprehension and vocabulary.",
    "geography": "For geography: Visualize maps while studying to better recall locations."
}

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')  # Ensure you've set GEMINI_API_KEY
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')  # Ensure you've set TAVILY_API_KEY

# Initialize Tavily client
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

# Initialize the Gemini LLM (Google Generative AI)
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=GEMINI_API_KEY
)

tools = []

class State(TypedDict):
    messages: list
    memory: list

MAX_MEMORY_LENGTH = 5

def truncate_memory(memory):
    if len(memory) > MAX_MEMORY_LENGTH:
        memory = memory[-MAX_MEMORY_LENGTH:]
    return memory

def summarize_memory(memory):
    # Summarize the conversation in a "You asked... I replied..." format
    user_assistant_pairs = []
    user_msg = None
    for turn in memory:
        if turn["role"] == "user":
            user_msg = turn["message"]
        elif turn["role"] == "assistant" and user_msg is not None:
            assistant_msg = turn["message"]
            user_assistant_pairs.append((user_msg, assistant_msg))
            user_msg = None

    if not user_assistant_pairs:
        return "No previous conversation found."

    summary_lines = []
    for (u, a) in user_assistant_pairs:
        summary_lines.append(f"You asked: '{u}'\nI replied: '{a}'")

    return "\n\n".join(summary_lines)

def chatbot(state: State):
    user_input = state["messages"][-1][1].strip()
    lower_input = user_input.lower()

    # Append current user message to memory
    state['memory'].append({"role": "user", "message": user_input})
    state['memory'] = truncate_memory(state['memory'])

    # If the user asks who made you
    if "who made you" in lower_input:
        response = "I was created by Muhammad bin Zohaib."
        state['memory'].append({"role": "assistant", "message": response})
        state['memory'] = truncate_memory(state['memory'])
        return {"messages": [response], "memory": state["memory"]}

    # Check for memory summary
    if "show memory" in lower_input:
        summary = summarize_memory(state['memory'])
        state['memory'].append({"role": "assistant", "message": summary})
        state['memory'] = truncate_memory(state['memory'])
        return {"messages": [summary], "memory": state["memory"]}

    if "quit" in lower_input:
        quote = get_motivational_quote()
        state['memory'].append({"role": "assistant", "message": quote})
        state['memory'] = truncate_memory(state['memory'])
        return {"messages": [quote], "memory": state["memory"], "end": True}

    if "joke" in lower_input:
        # User wants a Q&A style joke
        joke_qa = get_joke_qa()
        state['memory'].append({"role": "assistant", "message": joke_qa})
        state['memory'] = truncate_memory(state['memory'])
        return {"messages": [joke_qa], "memory": state["memory"]}

    if "quote" in lower_input:
        quote = get_quote()
        state['memory'].append({"role": "assistant", "message": quote})
        state['memory'] = truncate_memory(state['memory'])
        return {"messages": [quote], "memory": state["memory"]}

    if "what did i ask before" in lower_input:
        previous_user_inputs = [m["message"] for m in state['memory'] if m["role"] == "user"]
        if len(previous_user_inputs) > 1:
            last_input = previous_user_inputs[-2]
            response = f"You previously asked: '{last_input}'."
        else:
            response = "You haven't asked anything before this."
        state['memory'].append({"role": "assistant", "message": response})
        state['memory'] = truncate_memory(state['memory'])
        return {"messages": [response], "memory": state["memory"]}

    if "research on this" in lower_input:
        context = tavily_client.get_search_context(query=user_input)
        answer = tavily_client.qna_search(query=user_input)

        url_link = None
        if isinstance(context, list) and len(context) > 0:
            first_item = context[0]
            if isinstance(first_item, str):
                try:
                    c_obj = json.loads(first_item)
                    url_link = c_obj.get("url", None)
                except:
                    url_link = None
            elif isinstance(first_item, dict):
                url_link = first_item.get("url", None)

        if url_link:
            formatted_answer = f"Here's some research information I found:\n\n{answer}\n\nFor more details, check this link: {url_link}"
        else:
            formatted_answer = f"Here's some research information I found:\n\n{answer}"

        state['memory'].append({"role": "assistant", "message": formatted_answer})
        state['memory'] = truncate_memory(state['memory'])
        return {"messages": [formatted_answer], "memory": state['memory']}

    if "study tip" in lower_input:
        tip = get_study_tip()
        state['memory'].append({"role": "assistant", "message": tip})
        state['memory'] = truncate_memory(state['memory'])
        return {"messages": [tip], "memory": state['memory']}

    if "fact" in lower_input:
        fact = get_fact()
        state['memory'].append({"role": "assistant", "message": fact})
        state['memory'] = truncate_memory(state['memory'])
        return {"messages": [fact], "memory": state['memory']}

    # Check subject-specific prompts
    for subject, prompt in subject_prompts.items():
        if subject in lower_input:
            response = prompt
            state['memory'].append({"role": "assistant", "message": response})
            state['memory'] = truncate_memory(state['memory'])
            return {"messages": [response], "memory": state['memory']}

    # Otherwise, use Gemini for a conversational or academic response
    conversation = []
    for turn in state['memory']:
        conversation.append((turn["role"], turn["message"]))

    response = llm.invoke(conversation)

    if hasattr(response, 'content'):
        response = response.content

    formatted_response = response.strip() if isinstance(response, str) else str(response)
    if not formatted_response:
        formatted_response = "I'm here to help with your studies. Could you clarify what you need help with today?"

    state['memory'].append({"role": "assistant", "message": formatted_response})
    state['memory'] = truncate_memory(state['memory'])

    return {"messages": [formatted_response], "memory": state['memory']}

# Set up the graph
graph_builder = StateGraph(State)
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

def run_agent():
    greeting = (
        "👋 Hello! I'm StudyBuddy, your dedicated study assistant.\n"
        "📚 I can help with exam preparation, study materials, research assistance (just say 'research on this'), \n   Jokes (Q&A style), quotes, study tips, facts, and subject-specific encouragement."
    )
    print(greeting)

    memory = []
    while True:
        user_input = input("📥 You: ")
        events = graph.stream(
            {"messages": [("user", user_input)], "memory": memory},
            stream_mode="values"
        )

        end_chat = False
        for event in events:
            print(event['messages'][-1])
            memory = event["memory"]
            if "end" in event:
                end_chat = True

        if end_chat:
            print("Goodbye! Keep learning and growing. 📚✏️")
            break

run_agent()


👋 Hello! I'm StudyBuddy, your dedicated study assistant.
📚 I can help with exam preparation, study materials, research assistance (just say 'research on this'), 
   Jokes (Q&A style), quotes, study tips, facts, and subject-specific encouragement.


KeyboardInterrupt: Interrupted by user